<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/Loading_customized_data_images_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLaB with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please load your drive....")
  COLAB = False
def time_fmt(t:float = 123.1093)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}:{s:>05.2f}"
print(f">>>testing the function:\tplease wait...\n>>>time elapse:\t{time_fmt()}")

Mounted at /content/drive
You are on CoLaB with tensorflow version: 2.4.1
>>>testing the function:	please wait...
>>>time elapse:	0: 02:03.00


In [ ]:
#In this notebook we will discuss several ways of loading customized datasets/images 

In [ ]:
import tensorflow as tf
import numpy as np
import time, os, sys 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#loading data from the directory using keras preprocessing:

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/transfer',
    labels = 'inferred',
    label_mode = 'int',
    class_names = ['cd','disk35','disk525','tape'],
    shuffle = True,
    seed = 119,
    validation_split = 0.1, subset = 'training'
)

validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/transfer',
    labels = 'inferred',
    label_mode = 'int',
    class_names = ['cd','disk35','disk525','tape'],
    shuffle = True,
    seed = 119,
    validation_split = 0.1, subset = 'validation'
)

Found 52 files belonging to 4 classes.
Using 47 files for training.
Found 52 files belonging to 4 classes.
Using 5 files for validation.


In [ ]:
#Examine the shape of the data
x_batch, y_batch = next(iter(train_data))

In [ ]:
x_batch.shape

TensorShape([32, 256, 256, 3])

In [ ]:
#we can do some data augumentation:

In [ ]:
def _img_augument(image, label):
  image = tf.image.random_brightness(image, max_delta = 0.2)
  image = tf.image.flip_left_right(image)
  image = tf.image.flip_up_down(image)
  return image, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_data = train_data.map(_img_augument, num_parallel_calls= AUTOTUNE)

In [ ]:
inputs = tf.keras.Input(shape = (256,256,3), name = 'inputs_shape')
x = tf.keras.layers.Conv2D(64,3,padding = 'same', activation = 'relu')(inputs)
x = tf.keras.layers.Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(256, 3, padding = 'same', activation  = 'relu')(x)
x = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(units = 128, activation = 'relu')(x)
x = tf.keras.layers.Dense(units = 64, activation="relu")(x)
outputs = tf.keras.layers.Dense(units = 4, activation = 'softmax')(x)
model = tf.keras.Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs_shape (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 128)     73856     
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 128)     512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 256)     295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 512)     118016

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'Adam',
              metrics = ['accuracy'])

In [ ]:
model.fit(train_data, validation_data = validation_data, epochs = 3, verbose = 2)

Epoch 1/3
2/2 - 176s - loss: 1.3382 - accuracy: 0.4255 - val_loss: 23.9672 - val_accuracy: 0.0000e+00
Epoch 2/3
2/2 - 167s - loss: 1.0569 - accuracy: 0.5319 - val_loss: 32.1468 - val_accuracy: 0.0000e+00
Epoch 3/3
2/2 - 166s - loss: 0.8778 - accuracy: 0.5957 - val_loss: 54.5133 - val_accuracy: 0.0000e+00


In [14]:
#We can also load data from the directory using imagedata generator:

In [16]:
datagen = ImageDataGenerator(
    rescale = 1.0/255.0,
    rotation_range = 5,
    zoom_range = (0.95, 0.95),
    horizontal_flip = False,
    vertical_flip = False,
    dtype = tf.float32,
    data_format = 'channels_last',
    validation_split = 0.0    
)


In [17]:
train_dfm = datagen.flow_from_directory(
   directory = "/content/drive/MyDrive/transfer",
   target_size = (256,256),
   color_mode = 'grayscale',
   class_mode = 'sparse',
   shuffle = True,
   subset = 'training'

    
)

Found 52 images belonging to 4 classes.
